In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv
/kaggle/input/distil_bert/keras/distil_bert_base_en_uncased/1/config.json
/kaggle/input/distil_bert/keras/distil_bert_base_en_uncased/1/tokenizer.json
/kaggle/input/distil_bert/keras/distil_bert_base_en_uncased/1/metadata.json
/kaggle/input/distil_bert/keras/distil_bert_base_en_uncased/1/model.weights.h5
/kaggle/input/distil_bert/keras/distil_bert_base_en_uncased/1/assets/tokenizer/vocabulary.txt
/kaggle/input/llm-mistral-7b-instruct-texts/Mistral7B_CME_v3.csv
/kaggle/input/llm-mistral-7b-instruct-texts/Mistral7B_CME_v5.csv
/kaggle/input/llm-mistral-7b-instruct-texts/Mistral7B_CME_v7.csv
/kaggle/input/llm-mistral-7b-instruct-texts/Mistral7B_CME_v6.csv
/kaggle/input/llm-mistral-7b-instruct-texts/Mistral7B_CME_v2.csv
/kaggle/input/llm-mistral-7b-instruct-texts/Mistral7B_CME_v1.csv
/kaggle/input/llm-mistral-7b-instruct-texts/Mistral7B_CME_v7_15_percent_corruption.csv
/kaggle/input/llm-mistral-7b-instruct-texts/Mistral7B_CME_v4.csv

In [2]:
data_path="/kaggle/input/llm-detect-ai-generated-text/"
sample_submission=pd.read_csv(data_path+'sample_submission.csv')
train_prompts=pd.read_csv(data_path+'train_prompts.csv')
test_essays =pd.read_csv(data_path+"test_essays.csv")
train_essays = pd.read_csv(data_path+"train_essays.csv")

In [3]:
print(sample_submission.info())
print(sample_submission.head)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         3 non-null      object 
 1   generated  3 non-null      float64
dtypes: float64(1), object(1)
memory usage: 176.0+ bytes
None
<bound method NDFrame.head of          id  generated
0  0000aaaa        0.1
1  1111bbbb        0.9
2  2222cccc        0.4>


In [4]:
print(train_prompts.info())
print(train_prompts.head)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   prompt_id     2 non-null      int64 
 1   prompt_name   2 non-null      object
 2   instructions  2 non-null      object
 3   source_text   2 non-null      object
dtypes: int64(1), object(3)
memory usage: 192.0+ bytes
None
<bound method NDFrame.head of    prompt_id                       prompt_name  \
0          0                   Car-free cities   
1          1  Does the electoral college work?   

                                        instructions  \
0  Write an explanatory essay to inform fellow ci...   
1  Write a letter to your state senator in which ...   

                                         source_text  
0  # In German Suburb, Life Goes On Without Cars ...  
1  # What Is the Electoral College? by the Office...  >


In [5]:
print(train_essays.info())
print(train_essays.head)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1378 entries, 0 to 1377
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         1378 non-null   object
 1   prompt_id  1378 non-null   int64 
 2   text       1378 non-null   object
 3   generated  1378 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 43.2+ KB
None
<bound method NDFrame.head of             id  prompt_id                                               text  \
0     0059830c          0  Cars. Cars have been around since they became ...   
1     005db917          0  Transportation is a large necessity in most co...   
2     008f63e3          0  "America's love affair with it's vehicles seem...   
3     00940276          0  How often do you ride in a car? Do you drive a...   
4     00c39458          0  Cars are a wonderful thing. They are perhaps o...   
...        ...        ...                                                ...   
1373  fe6ff9a5

In [6]:
print(test_essays.info())
print(test_essays.head)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         3 non-null      object
 1   prompt_id  3 non-null      int64 
 2   text       3 non-null      object
dtypes: int64(1), object(2)
memory usage: 200.0+ bytes
None
<bound method NDFrame.head of          id  prompt_id          text
0  0000aaaa          2  Aaa bbb ccc.
1  1111bbbb          3  Bbb ccc ddd.
2  2222cccc          4  CCC ddd eee.>


In [7]:
train_essays["generated"].value_counts()

generated
0    1375
1       3
Name: count, dtype: int64

In [8]:
train_essays["prompt_id"].value_counts()

prompt_id
0    708
1    670
Name: count, dtype: int64

In [9]:
oth_dp="/kaggle/input/llm-mistral-7b-instruct-texts/Mistral7B_CME_v7.csv"
dataset1=pd.read_csv(oth_dp)

In [10]:
dataset1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4900 entries, 0 to 4899
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   prompt_id    4900 non-null   int64 
 1   text         4900 non-null   object
 2   prompt_name  4900 non-null   object
 3   generated    4900 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 153.2+ KB


In [11]:
columns_to_delete = ['prompt_id','id']
train_essays.drop(columns=columns_to_delete, inplace=True)
columns_to_delete = ['prompt_id','prompt_name']
dataset1.drop(columns=columns_to_delete, inplace=True)

In [12]:
train_essays=pd.concat([train_essays,dataset1],ignore_index=True)

In [13]:
train_essays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6278 entries, 0 to 6277
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       6278 non-null   object
 1   generated  6278 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 98.2+ KB


In [14]:
train_essays["generated"].value_counts()

generated
1    4903
0    1375
Name: count, dtype: int64

In [15]:
oth_dp="/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv"
dataset2=pd.read_csv(oth_dp)
dataset2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44868 entries, 0 to 44867
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   text           44868 non-null  object
 1   label          44868 non-null  int64 
 2   prompt_name    44868 non-null  object
 3   source         44868 non-null  object
 4   RDizzl3_seven  44868 non-null  bool  
dtypes: bool(1), int64(1), object(3)
memory usage: 1.4+ MB


In [16]:
dataset2 = dataset2[dataset2.RDizzl3_seven]
columns_to_delete = ['prompt_name','source','RDizzl3_seven']
dataset2.drop(columns=columns_to_delete, inplace=True)
dataset2=dataset2.rename(columns={
    "text":"text",
    "label":"generated"
})

In [17]:
train_essays=pd.concat([train_essays,dataset2],ignore_index=True)

In [18]:
# oth_dp="/kaggle/input/daigt-proper-train-dataset/train_drcat_04.csv"
# dataset3=pd.read_csv(oth_dp)
# dataset3.info()

In [19]:
# columns_to_delete = ['essay_id','source','prompt','fold']
# dataset3.drop(columns=columns_to_delete, inplace=True)
# dataset3=dataset3.rename(columns={
#     "text":"text",
#     "label":"generated"
# })
# dataset3.info()

In [20]:
# train_essays=pd.concat([train_essays,dataset3],ignore_index=True)

In [21]:
train_essays=train_essays.sample(frac=1, random_state=56)

In [22]:
train_essays.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26728 entries, 21480 to 2532
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       26728 non-null  object
 1   generated  26728 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 626.4+ KB


In [23]:
train_essays["generated"].value_counts()

generated
0    15625
1    11103
Name: count, dtype: int64

In [24]:
from tokenizers import (decoders,models,normalizers,pre_tokenizers,processors,trainers,Tokenizer)
from tqdm.auto import tqdm
from datasets import Dataset
from transformers import PreTrainedTokenizerFast

In [25]:

LOWERCASE = False
VOCAB_SIZE = 3052
raw_tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))
raw_tokenizer.normalizer = normalizers.Sequence([normalizers.NFC()] + [normalizers.Lowercase()] if LOWERCASE else [])
raw_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.BpeTrainer(vocab_size=VOCAB_SIZE, special_tokens=special_tokens)

# Train the tokenizer on extra_train
dataset_extra_train = Dataset.from_pandas(train_essays[['text']])
def train_corp_iter(): 
    for i in range(0, len(dataset_extra_train), 1000):
        yield dataset_extra_train[i : i + 1000]["text"]
raw_tokenizer.train_from_iterator(train_corp_iter(), trainer=trainer)

tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=raw_tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)

final_test = []
for text in tqdm(test_essays['text'].tolist()):
    final_test.append(tokenizer.tokenize(text))

# Tokenization of Extra Train Texts
final_train = []
for text in tqdm(train_essays['text'].tolist()):
    final_train.append(tokenizer.tokenize(text))

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/26728 [00:00<?, ?it/s]

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
def dummy(text):
    return text

# Fit the vectorizer on test data
vectorizer = TfidfVectorizer(
    ngram_range=(3, 5),
    lowercase=False,
    sublinear_tf=True,
    analyzer='word',
    tokenizer=dummy,
    preprocessor=dummy,
    token_pattern=None,
    strip_accents='unicode'
)

vectorizer.fit(final_test)

# Getting vocab
vocab = vectorizer.vocabulary_
# Create a new vectorizer using the vocabulary obtained from test data
vectorizer = TfidfVectorizer(
    ngram_range=(3, 5),
    lowercase=False,
    sublinear_tf=True,
    vocabulary=vocab,
    analyzer='word',
    tokenizer=dummy,
    preprocessor=dummy,
    token_pattern=None,
    strip_accents='unicode'
)

# Fit and transform extra_train and test data
x = vectorizer.fit_transform(final_train)
test_transform = vectorizer.transform(final_test)

# Cleanup
import gc
del vectorizer
gc.collect()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


23

In [27]:
y=train_essays.generated

In [28]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.linear_model import SGDClassifier
# from sklearn.ensemble import VotingClassifier


# # Training classifier model
# # Logistic Regression model
# lr = LogisticRegression(verbose=1)

# # Multinomial Naive Bayes model
# clf = MultinomialNB(alpha=0.02)

# # SGDClassifier models with different hyperparameters
# sgd_model = SGDClassifier(max_iter=8000, tol=1e-3, loss="modified_huber")
# sgd_model2 = SGDClassifier(max_iter=12000, tol=5e-4, loss="modified_huber", class_weight="balanced")
# sgd_model3 = SGDClassifier(max_iter=15000, tol=3e-4, loss="modified_huber", early_stopping=True)

# # Ensemble VotingClassifier
# final_model = VotingClassifier(
#     estimators=[('lr', lr), ('mnb', clf), ('sgd', sgd_model), ('sgd2', sgd_model2), ('sgd3', sgd_model3)],
#     weights=[0.20, 0.20, 0.20, 0.20, 0.20],
#     voting='soft',
#     verbose=1
# )

# final_model.fit(x,y)

In [29]:
#from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


clf = MultinomialNB(alpha=0.02)
sgd_model = SGDClassifier(max_iter=8000, tol=1e-4, loss="modified_huber") 
p6 = {
        'n_iter': 1500,
        'verbose': -1,
        'objective': 'binary',
        'metric': 'auc',
        'learning_rate': 0.05073909898961407,
        'colsample_bytree': 0.726023996436955,
        'colsample_bynode': 0.5803681307354022,
        'lambda_l1': 8.562963348932286,
        'lambda_l2': 4.893256185259296,
        'min_data_in_leaf': 115,
        'max_depth': 23,
        'max_bin': 898
    }
lgb = LGBMClassifier(**p6)
cat = CatBoostClassifier(
        iterations=1000,
        verbose=0,
        l2_leaf_reg=6.6591278779517808,
        learning_rate=0.005689066836106983/2,
        allow_const_label=True,
        loss_function = 'CrossEntropy',
        random_seed=1234
    )
    
weights = [0.07, 0.41, 0.41, 0.41]
final_model = VotingClassifier(
        estimators=[
            ('mnb', clf),
            ('sgd', sgd_model),
            ('lgb', lgb), 
            ('cat', cat)
        ],
        weights=weights, voting='soft', n_jobs=-1
    )

final_model.fit(x,y)


/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/

VotingClassifier(estimators=[('mnb', MultinomialNB(alpha=0.02)),
                             ('sgd',
                              SGDClassifier(loss='modified_huber',
                                            max_iter=8000, tol=0.0001)),
                             ('lgb',
                              LGBMClassifier(colsample_bynode=0.5803681307354022,
                                             colsample_bytree=0.726023996436955,
                                             lambda_l1=8.562963348932286,
                                             lambda_l2=4.893256185259296,
                                             learning_rate=0.05073909898961407,
                                             max_bin=898, max_depth=23,
                                             metric='auc', min_data_in_leaf=115,
                                             n_iter=1500, objective='binary',
                                             verbose=-1)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x7d71ce725ed0>)],
                 n_jobs=-1, voting='soft', weights=[0.07, 0.41, 0.41, 0.41])

In [30]:
y_probs = final_model.predict_proba(test_transform)
y_probs

array([[0.59420745, 0.40579255],
       [0.59347846, 0.40652154],
       [0.58522331, 0.41477669]])

In [31]:
y_gen_prob =[y[1] for y in y_probs]
y_gen_prob

[0.40579254718044167, 0.40652153906587407, 0.41477669466419154]

In [32]:
sub = test_essays[["id"]].copy()

sub["generated"] = y_gen_prob

# Save Submission
sub.to_csv('submission.csv',index=False)

sub.head()

,id,generated
0,0000aaaa,0.405793
1,1111bbbb,0.406522
2,2222cccc,0.414777


0	0000aaaa	0.539238
1	1111bbbb	0.570501
2	2222cccc	0.570501